## Video Load

In [ ]:
# !pip install opencv-python
# !pip install matplotlib==3.3.0

In [9]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [10]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            cv2.imshow("video", frame)
            cv2.waitKey(ms) # Millisecond 단위로 대기
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [6]:
# Video가 저장된 경로 입력
PATH = r".\pushup_sample.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [11]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
if video.size > 0:
    print(f"- Maximum Intensity: {video.max()}")
    print(f"- Minimum Intensity: {video.min()}")
else:
    print("The array is empty, so maximum and minimum intensities cannot be determined.")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (2023, 360, 640, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 255
- Minimum Intensity: 0


In [34]:
import cv2
import numpy as np

def preprocess_frame(frame):
    # 그레이스케일로 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # 블러링 처리로 노이즈 제거
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)

    # 원형 커널 정의 (반지름 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(blurred, cv2.MORPH_OPEN, kernel)

    plt.imshow(opening, cmap='gray')
    plt.show()
    return opening

def detect_edges(frame):
    # 캐니 엣지 검출
    edges = cv2.Canny(frame, 50, 150)
    return edges

def find_contours(edges):
    # 윤곽선 검출
    contours, _ = cv2.findContours(edges.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return contours

def detect_head(image):
    # 이미지를 HSV 색 공간으로 변환
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # 피부색에 해당하는 HSV 범위 설정
    lower_skin = np.array([0, 20, 70], dtype=np.uint8)
    upper_skin = np.array([20, 255, 255], dtype=np.uint8)
    
    # 피부색 영역을 마스크로 생성
    mask = cv2.inRange(hsv, lower_skin, upper_skin)
    
    # 모폴로지 연산을 사용하여 노이즈 제거
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    
    # 윤곽선 검출
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # 가장 큰 윤곽선을 찾기
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        # 윤곽선을 둘러싸는 외접 원을 계산
        ((x, y), radius) = cv2.minEnclosingCircle(largest_contour)
        center = (int(x), int(y))
        radius = int(radius)
        
        # 머리 부분으로 간주하여 원 그리기
        if radius > 10:  # 최소 반지름 조건
            cv2.circle(image, center, radius, (0, 255, 0), 2)
            cv2.putText(image, 'Head', (center[0] - 10, center[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return image, mask

def detect_body_parts(frame, contours):
    # 신체 부위를 인식하기 위한 간단한 방법 (최대 윤곽선)
    if len(contours) == 0:
        return None, None
    
    # 가장 큰 윤곽선 선택
    max_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(max_contour)
    
    # 머리와 어깨 위치 추정
    head = (x + w // 2, y)
    shoulder = (x + w // 2, y + h // 4)
    
    return head, shoulder

def count_pushups(head_positions):
    count = 0
    state = 0  # 0: 시작, 1: 내려감, 2: 올라옴
    for i in range(1, len(head_positions)):
        if head_positions[i-1] is None or head_positions[i] is None:
            continue
        if state == 0 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
        elif state == 1 and head_positions[i][1] < head_positions[i-1][1]:
            count += 1
            state = 2
        elif state == 2 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
    return count

def detect_skin(img):

    #converting from gbr to hsv color space
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #skin color range for hsv color space 
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 
    HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #converting from gbr to YCbCr color space
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    #skin color range for hsv color space 
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #merge skin detection (YCbCr and hsv)
    global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
    global_mask=cv2.medianBlur(global_mask,3)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((4,4), np.uint8))


    HSV_result = cv2.bitwise_not(HSV_mask)
    YCrCb_result = cv2.bitwise_not(YCrCb_mask)
    global_result=cv2.bitwise_not(global_mask)

    return global_result

def get_largest_contour(mask):
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        return largest_contour
    return None

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    head_positions = []
    frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # 피부색 검출
        mask = detect_skin(frame)

        # 결과 이미지 출력
        cv2.imshow('Original', mask)
        

        frames.append(frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    pushup_count = count_pushups(head_positions)
    print(f"Total push-ups: {pushup_count}")

if __name__ == "__main__":
    video_path = "pushup_video.mp4"
    main(PATH)

Total push-ups: 0
